In [13]:
# model and X_test, y_test, etc. are defined by this script
from credoai.datasets import fetch_credit_model

# model and data are defined by this script
X_test, y_test, sensitive_features_test, model = fetch_credit_model()

In [14]:
X_test

,LIMIT_BAL,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,...,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6
2308,30000.0,2.0,2.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,...,11581.0,12580.0,13716.0,14828.0,1500.0,2000.0,1500.0,1500.0,1500.0,2000.0
22404,150000.0,1.0,2.0,26.0,0.0,0.0,0.0,0.0,0.0,0.0,...,116684.0,101581.0,77741.0,77264.0,4486.0,4235.0,3161.0,2647.0,2669.0,2669.0
23397,70000.0,3.0,1.0,32.0,0.0,0.0,0.0,0.0,0.0,0.0,...,68530.0,69753.0,70111.0,70212.0,2431.0,3112.0,3000.0,2438.0,2500.0,2554.0
25058,130000.0,3.0,2.0,49.0,0.0,0.0,0.0,0.0,0.0,-1.0,...,16172.0,16898.0,11236.0,6944.0,1610.0,1808.0,7014.0,27.0,7011.0,4408.0
2664,50000.0,2.0,2.0,36.0,0.0,0.0,0.0,0.0,0.0,2.0,...,42361.0,19574.0,20295.0,19439.0,2000.0,1500.0,1000.0,1800.0,0.0,1000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3211,40000.0,3.0,1.0,49.0,1.0,-2.0,-2.0,-2.0,-2.0,-1.0,...,0.0,0.0,0.0,150.0,0.0,0.0,0.0,0.0,150.0,159.0
9355,330000.0,2.0,1.0,42.0,0.0,0.0,0.0,0.0,0.0,0.0,...,32912.0,36710.0,40442.0,42283.0,10000.0,10000.0,5000.0,5000.0,5000.0,5000.0
28201,300000.0,3.0,2.0,48.0,1.0,3.0,2.0,-1.0,0.0,-1.0,...,390.0,769.0,772.0,1722.0,781.0,1.0,769.0,3.0,1730.0,0.0
19705,100000.0,2.0,1.0,47.0,-1.0,-1.0,-1.0,-1.0,-1.0,-2.0,...,0.0,6270.0,-396.0,-396.0,6666.0,0.0,6270.0,0.0,0.0,0.0


In [15]:
y_test

2308     0
22404    0
23397    0
25058    0
2664     1
        ..
3211     0
9355     0
28201    1
19705    1
28313    0
Name: target, Length: 7500, dtype: int64

In [9]:
from credoai.lens import Lens
from credoai.artifacts import ClassificationModel, TabularData
from credoai.evaluators import ModelFairness, Performance

In [12]:
credo_model = ClassificationModel(name="credit_default_classifier", model_like=model)
credo_data = TabularData(
    name="UCI-credit-default",
    X=X_test,
    y=y_test,
    sensitive_features=sensitive_features_test,
)
lens = Lens(model=credo_model, assessment_data=credo_data)
metrics = ['precision_score', 'recall_score', 'equal_opportunity']
lens.add(ModelFairness(metrics=metrics))
lens.add(Performance(metrics=metrics))
lens.run()

2023-04-12 18:45:15,483 - lens - INFO - Evaluator ModelFairness added to pipeline. Sensitive feature: SEX
2023-04-12 18:45:15,750 - lens - INFO - Evaluator Performance added to pipeline. 
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s finished


In [16]:
results = lens.get_results()
print(f"Results for {len(results)} evaluators")

Results for 2 evaluators


In [17]:
performance_results = lens.get_results(evaluator_name='Performance')[0]
# first dataframe is overall metrics
display(performance_results['results'][0])
# second dataframe is the long form of the confusion matrix
display(performance_results['results'][1])

,type,value
0,precision_score,0.628081
1,recall_score,0.360172


,true_label,predicted_label,value
0,0,0,0.940916
1,1,0,0.639828
2,0,1,0.059084
3,1,1,0.360172
